In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import HyperModel
import pandas as pd

C:\Users\nptu\AppData\Local\Temp\ipykernel_17072\3212554359.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [7]:
class MyHyperModel(HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
        model.add(layers.Dense(1))
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
            loss='mean_squared_error',
            metrics=['mse'])
        return model

hypermodel = MyHyperModel()

In [3]:
BTC_all = pd.read_csv('BTC-all.csv').drop('Date',axis=1)
n = len(BTC_all)
y_train,y_val = BTC_all['Adj Close'][:int(n*0.7)],BTC_all['Adj Close'][int(n*0.7):]
x_train,x_val= BTC_all.drop('Adj Close',axis=1)[:int(n*0.7)],BTC_all.drop('Adj Close',axis=1)[int(n*0.7):]
print(x_train.shape,x_val.shape,y_train.shape,y_val.shape)


(2227, 14) (955, 14) (2227,) (955,)


In [9]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(
    hypermodel,
    objective='val_loss',
    max_trials=10,  # 要試驗的超參數組合數量
    directory='my_dir02',  # 儲存搜尋結果的目錄
    project_name='my_project'  # 這個搜尋的專案名稱
)

tuner.search(x_train,y_train , epochs=5, validation_data=(x_val, y_val))


Trial 10 Complete [00h 00m 02s]
val_loss: 77790640.0

Best val_loss So Far: 77790640.0
Total elapsed time: 00h 00m 19s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

In [ ]:
model = tuner.hypermodel.build(best_hps)
